# MOVIES BOX OFFICE REVIEW AND ANALYSIS.

This project is designed to  analyse the business side of movies to come up with the right solutions for a start up company in the movies space.

Specifically, this will cover:

* Financial analysis and correlations
* Gener descriptions and favourabilty
* Market and critics reviews

This is a comparative analysis for investors use.

## Data Understanding

In this repository we will be working with diffrent data sets including a sql database, CSV file and a TSV file
the are under  file path :
csv path "C:\\Users\\HP\\Desktop\\moringa\\PHASE1_PROJECT\\tn.movie_budgets.csv\\tn.movie_budgets.csv" and "C:\\Users\\HP\\Desktop\\moringa\\PHASE1_PROJECT\\bom.movie_gross.csv\\bom.movie_gross.csv"
 sql database "C:\Users\HP\Desktop\moringa\PHASE1_PROJECT\im.db\im.db"

  ## Objectives

* Perform Market analysis
* Perform financial analysis
* Review critics opinions

## Setup
Import the relevant libraries

In [ ]:
import pandas as pd
import matplotlib as plt
import sqlite3
import os
import seaborn as sns

## Importing and filtering data
importing relevant databases and Files for analysis and retriving necessary information to perform the required queries

In [ ]:
#retriing table names from data frame
im_path = r"C:\Users\HP\Desktop\moringa\PHASE1_PROJECT\im.db\im.db"

conn = sqlite3.connect(im_path)
cursor = conn.cursor()

table_name_query = """ SELECT name
                       AS 'Table Names' 
                       FROM sqlite_master 
                       WHERE type='table';
                       """

table_names = pd.read_sql(table_name_query, conn)

print(table_names)

In [ ]:
q = """SELECT *
       FROM directors;
    """
df = pd.read_sql(q, conn)

print(df.head())

## Merging CSV file
In the cells bellow i merge the two csv file in order to perform data cleaning. while also running functions so as to understand the data i am working with

In [ ]:
# marging the movie mudgets in csv file and  cleaning them up
budget_one = pd.read_csv("C:\\Users\\HP\\Desktop\\moringa\\PHASE1_PROJECT\\tn.movie_budgets.csv\\tn.movie_budgets.csv")
budget_two = pd.read_csv("C:\\Users\\HP\\Desktop\\moringa\\PHASE1_PROJECT\\bom.movie_gross.csv\\bom.movie_gross.csv")


In [ ]:
budget_one.head()
budget_one.tail()

In [ ]:
print(budget_one.columns)

In [ ]:
budget_one

In [ ]:
print(budget_two.columns)

In [ ]:
budget_two.drop(columns=["studio", "year"], inplace=True)
budget_two

In [ ]:
budget_two.columns = ['movie', 'domestic_gross', 'worldwide_gross']

# Print the updated DataFrame
print(budget_two)

In [ ]:
merged_revenue=pd.concat([budget_one, budget_two])
merged_revenue

## Data cleaning
After merging i clean the data and describe the values on the set.
Then perform basic mathe mathmatics to understand the values we are working with.
in this stage we replace the blanck value and NAM values with the median values since the max and the min are too extream, iam not using the average

In [ ]:
merged_revenue.drop_duplicates()

In [ ]:
merged_revenue['domestic_gross'] = pd.to_numeric(merged_revenue['domestic_gross'], errors='coerce')


In [ ]:
merged_revenue['domestic_gross'].mean()

In [ ]:
median_value=merged_revenue['domestic_gross'].median()
median_value

In [ ]:
merged_revenue['domestic_gross'].max()

In [ ]:
merged_revenue['domestic_gross'].describe()

In [ ]:
merged_revenue['domestic_gross'] = merged_revenue['domestic_gross'].fillna(median_value)

In [ ]:
merged_revenue['worldwide_gross'].describe()

In [ ]:
# im trying to remove the /$ signs 
merged_revenue['worldwide_gross'] = pd.to_numeric(merged_revenue['worldwide_gross'].replace('[\$,]', '', regex=True), errors='coerce')


In [ ]:
median=merged_revenue['worldwide_gross'].median()

In [ ]:
merged_revenue['worldwide_gross'] = merged_revenue['worldwide_gross'].fillna(median)

In [ ]:
merged_revenue['worldwide_gross'].describe()

## Importing the csv file on movies and data cleaning
At this point i imported the csv file with movie info and budget so as to start comparing the two data. 
after that i clean the data and try to understand the values and eplain them

In [ ]:
m1 = pd.read_csv("C:\\Users\\HP\\Desktop\\moringa\\PHASE1_PROJECT\\tn.movie_budgets.csv\\tn.movie_budgets.csv")
m1

## Transforming Data

Created a new row from m1 file so that i could merge it with the merged_revenue data

In [ ]:

production_budget = m1['production_budget']

production_budget = pd.to_numeric(production_budget.replace('[\$,]', '', regex=True), errors='coerce')
production_budget

In [ ]:
Release_year = m1['release_date']
Release_year

In [ ]:
merged_revenue['Release_year'] = Release_year

In [ ]:
production_budget.describe()

In [ ]:
merged_revenue
pd.set_option('display.float_format', '{:.2f}'.format)
merged_revenue

In [ ]:
merged_revenue['production_budget'] = production_budget
merged_revenue

In [ ]:
merged_revenue['production_budget'].describe()

In [ ]:
merged_revenue['production_budget'] = pd.to_numeric(merged_revenue['production_budget'].replace('[\$,]', '', regex=True), errors='coerce')

In [ ]:
median_production=merged_revenue['production_budget'].median()

In [ ]:
merged_revenue['production_budget'] = merged_revenue['production_budget'].fillna(median_production)

In [ ]:
merged_revenue['Total_revenue'] = merged_revenue['domestic_gross'] + merged_revenue['worldwide_gross']
merged_revenue['Total_revenue'] 

In [ ]:
merged_revenue['ROI'] = ((merged_revenue['Total_revenue'] - merged_revenue['production_budget']) / merged_revenue['production_budget']) * 100
merged_revenue['ROI']

In [ ]:
merged_revenue['domestic_gross'] = merged_revenue['domestic_gross'].round().astype(int)

merged_revenue['worldwide_gross'] = merged_revenue['worldwide_gross'].round().astype(int)

In [ ]:
merged_revenue

In [ ]:
import matplotlib.pyplot as plt

print(plt.figure)

## Data Visualization

The below cell we get to see how the data interact with diffrent values.
i did the visualizations with bar line and plots

### Return On Investment
I calculated the ROI and ploted a line graph to illustate its changes over the year. the data used is for the first 50 movie

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
merged_revenue['release_date'] = pd.to_datetime(merged_revenue['release_date'], errors='coerce')

# Sorting the DataFrame by 'release_date' to ensure the plot is in chronological order
merged_revenue = merged_revenue.sort_values(by='release_date')

# Plotting ROI over time for the first 50 rows using a line graph
plt.figure(figsize=(12, 6))
plt.plot(merged_revenue['release_date'].head(50), merged_revenue['ROI'].head(50), marker='o', linestyle='-', color='b')
plt.title('Return on Investment (ROI) Over Time For The First 50 Movies', fontsize=16)
plt.xlabel('Release Date', fontsize=12)
plt.ylabel('ROI (%)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

A descriptive analysis on the new data the relation between production budget,total revenue and ROI

In [ ]:
# finding out what the data looks like
descriptive_analysis= merged_revenue[['production_budget', 'Total_revenue', 'ROI']].describe()
print(descriptive_analysis)


## Correlation analysis
I calculated the correlation between production budgetand the total revenue
the results are as shown below. there is a positive corelation between the two

In [ ]:
# finding the corelation between production budget and total revenue returned(random samples)
random_sample = merged_revenue.sample(n=200, random_state=42) 

plt.figure(figsize=(10, 6))
sns.scatterplot(x='production_budget', y='Total_revenue', data=random_sample, hue='Total_revenue', palette='viridis', alpha=0.7, marker='o')
plt.title('Random Sample: Movie Budget vs Gross Returns', fontsize=16)
plt.xlabel('Production Budget (in millions)', fontsize=12)
plt.ylabel('Total Revenue (in millions)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)
sns.despine()
plt.legend(title='Total Revenue', bbox_to_anchor=(1, 1))
plt.show()

## Regrassion analysis

In [ ]:
# linear regression analysisi
plt.figure(figsize=(10, 6))
sns.regplot(x='production_budget', y='Total_revenue', data=random_sample, scatter_kws={'alpha':0.7, 's': 30}, line_kws={'color': 'red'})
plt.title('Random Sample: Production Budget and Gross Returns Linear Regression Analysis', fontsize=16)
plt.xlabel('Production Budget (in millions)', fontsize=12)
plt.ylabel('Total Revenue (in millions)', fontsize=12)
plt.grid(True, linestyle='--', alpha=0.5)
sns.despine()
plt.show()

## Market analysis

This was done to see how diffrent movies performed in the box office by compearing the budget of the movies and the revenues

In [ ]:
#  budget categories by definations
budget_bins = [20000000, 50000000, 100000000, float('inf')]  
budget_labels = ['Low Budget', 'Medium Budget', 'High Budget']

# Created a new column 'Budget_Category' based on budget ranges
merged_revenue['Budget_Category'] = pd.cut(merged_revenue['production_budget'], bins=budget_bins, labels=budget_labels, right=False)

plt.figure(figsize=(10, 6))
sns.countplot(x='Budget_Category', data=merged_revenue, palette='viridis')
plt.title('Distribution of Movies in Budget Categories')
plt.xlabel('Budget Category')
plt.ylabel('Number of Movies')
plt.show()

# analysis on revenue and reviews by budget category
plt.figure(figsize=(14, 6))

plt.subplot(1, 2, 1)
sns.boxplot(x='Budget_Category', y='Total_revenue', data=merged_revenue, palette='muted')
plt.title('Box Office Revenue Distribution by Budget Category')
plt.xlabel('Budget Category')
plt.ylabel('Total Revenue (in millions)')

plt.tight_layout()
plt.show()

A review of the best performing movies on the box office by total revenue return

In [ ]:
top_gross_return = merged_revenue.sort_values(by='Total_revenue', ascending=False).head(10)
top_gross_return 


And the lowest performing by total revenue

In [ ]:
low_gross_return = merged_revenue.sort_values(by='Total_revenue', ascending=False).tail(10)
low_gross_return

In [ ]:
# correlation between production cost and revenue return
correlation_budget_revenue = merged_revenue['production_budget'].corr(merged_revenue['Total_revenue'])
correlation_budget_revenue

## SQL Queries On im.db Database

I ran queries on the database on im.db to find out more information about the ovie industries the directors and writers contibution and there effects on returns


In [ ]:
Basic_info = '''
                 SELECT * FROM movie_basics;
'''

cursor.execute(Basic_info)

rows = cursor.fetchall()

for row in rows:
    print(row)

In [ ]:
Movies_release = '''
                 SELECT start_year, COUNT(*) AS movie_count
                 FROM movie_basics
                 GROUP BY start_year
                 ORDER BY start_year;
'''
cursor.execute(Movies_release)

rows = cursor.fetchall()

for row in rows:
    print(row)

In [ ]:
# executing sql queries on im.db
audience_Ratings_above_avg=''' 
                 SELECT *
                 FROM movie_ratings
                 WHERE averagerating >= 7.0;              
'''
cursor.execute(audience_Ratings_above_avg)

rows = cursor.fetchall()

for row in rows:
    print(row)


In [ ]:
geners_above_avg = '''
                 SELECT genres, AVG(averagerating) AS avg_rating
                 FROM movie_basics
                 JOIN movie_ratings ON movie_basics.movie_id = movie_ratings.movie_id
                 GROUP BY genres
                 HAVING avg_rating > 7.0;
'''

cursor.execute(geners_above_avg)

rows = cursor.fetchall()

for row in rows:
    print(row)



In [ ]:
Proffesions = '''
                 SELECT primary_name, primary_profession
                 FROM persons;
'''

cursor.execute(Proffesions)

rows = cursor.fetchall()

for row in rows:
    print(row)

In [ ]:
recognised_by = '''
                 SELECT persons.primary_name, known_for.movie_id
                 FROM persons
                 LEFT JOIN known_for ON persons.person_id = known_for.person_id;
'''

cursor.execute(recognised_by)

rows = cursor.fetchall()

for row in rows:
        print(row)


In [ ]:
English_movies= '''
                  SELECT title, region, language
                 FROM movie_akas
                 WHERE language = "en";
'''

cursor.execute(English_movies)

rows = cursor.fetchall()

for row in rows:
     print(row)
 

In [ ]:
critics_file = r'C:\\Users\\HP\\Desktop\\moringa\\PHASE1_PROJECT\\rt.reviews.tsv\\rt.reviews.tsv'

critics_reviews = pd.read_csv(critics_file, sep='\t', encoding='latin-1')

critics_reviews


In [ ]:
movies_info_file = r"C:\\Users\\HP\\Desktop\\moringa\\PHASE1_PROJECT\\rt.movie_info.tsv\\rt.movie_info.tsv"

movies_info = pd.read_csv(movies_info_file, sep='\t', encoding='latin-1')

movies_info

In [ ]:
movies_info.drop(columns=['synopsis', 'theater_date',	'dvd_date', 'currency', 'box_office'])

## Movies Distribution visualization
 
 I looked for the most in demand geners of movies.selecting the most performig geners to reommend for investors

In [ ]:
# Distribution of produced movies by geners
genre_counts = movies_info['genre'].value_counts()
genre_counts.head(10)

genre_counts.head(10).plot(kind='bar', edgecolor='black', rot=45, figsize=(15, 10))
plt.title('Distribution of Movies Across Genres')
plt.xlabel('genre_counts')
plt.ylabel('Number of Movies')
plt.show()

## Directors distribution

In [ ]:
# Distribution of Novies Produced by Directors
directors_counts = movies_info['director'].value_counts()
directors_counts.head(10)

plt.bar(directors_counts.index[:10], directors_counts.head(10).values, edgecolor='black', width=0.8)
plt.title('Number of Movies Directed by Each Director (Top 10)')
plt.xlabel('Director')
plt.ylabel('Number of Movies')
plt.xticks(rotation=45, ha='right', fontsize=8)
plt.tight_layout()
plt.show()

In [ ]:
writers_count=movies_info['writer'].value_counts()
writers_count

## Studios distribution

In [ ]:
# distribution by studios
studios_count=movies_info['studio'].value_counts()
studios_count.head(20)

studios_count.head(20).plot(kind='bar', edgecolor='black', figsize=(12, 6))
plt.title('Distribution of Movies Across Studios')
plt.xlabel('Studio')
plt.ylabel('Number of Movies')
plt.xticks(rotation=45, ha='right')  
plt.show()

In [ ]:
movies_info['runtime'].describe()

In [ ]:
critics_reviews['fresh'].value_counts()


## Critics opinion and ratings

This part i tried to find the critics opinions and there take on movies reviews

In [ ]:
critics_reviews['rating'].value_counts()

In [ ]:
critics_reviews['critic'].value_counts()

In [ ]:
critics_reviews['publisher'].value_counts()

In [ ]:
publishers_with_fresh_reviews = critics_reviews[critics_reviews['fresh'] == 'fresh']
publishers_with_fresh_reviews.head()

In [ ]:
publishers_with_rotten_reviews = critics_reviews[critics_reviews['fresh'] == 'rotten']
publishers_with_rotten_reviews